<h1 align="center">Grupo de permutaciones P(4)</h1>
<div align="right">David A. Miranda, PhD<br>2021</div>

## 1. Importar librerías

In [1]:
import numpy as np
import pandas as pd 

## 2. Planteamiento del problema

Obtener la tabla de multiplicación, las clases, subgrupos y los co-conjuntos derechos e izquierdos de los subgrupos para el grupo de permutaciones P(4), formado por elementos $g_k = (p_1, p_2, p_3, p_4)$, donde $p_i$ puede tomar los valores entre $1$ y $4$. 

El producto entre dos elemetos del grupo de permutaciones está dado por:

$$g_k g_{k'} = (p_1, p_2, p_3, p_4)(p'_1, p'_2, p'_3, p'_4)$$
$$g_k g_{k'} = (p'_{p_1}, p'_{p_2}, p'_{p_3}, p'_{p_4})$$

Donde $p'_{p_1}$ es valor en el índice $p_1$ del elemento $g_{k'}$ del grupo.

## 3. Definiciones y Métodos

In [2]:
P4 = {
    'e': [1,2,3,4],
    'a': [1,2,4,3],
    'b': [2,1,3,4],
    'c': [2,1,4,3],
    'd': [1,3,2,4],
    'f': [1,3,4,2]
}

In [3]:
P4.update({
    'g': [3,1,2,4],
    'h': [3,1,4,2],
    'i': [2,3,1,4],
    'j': [2,3,4,1],
    'k': [3,2,1,4],
    'l': [3,2,4,1]
})

In [4]:
P4.update({
    'm': [1,4,2,3],
    'n': [1,4,3,2],
    'o': [4,1,2,3],
    'p': [4,1,3,2],
    'q': [2,4,1,3],
    'r': [2,4,3,1]
})

In [5]:
P4.update({
    's': [4,2,1,3],
    't': [4,2,3,1],
    'u': [3,4,1,2],
    'v': [3,4,2,1],
    'w': [4,3,1,2],
    'y': [4,3,2,1]
})

In [6]:
def prod(gi, gj, group):
    """ 
    prod(gi, gj, group) retorna el producto gi gj de los elementos etiquetados como 
    gi y gj para el grupo group.  Tanto gi como gj corresponden a etiquetas (string)
    y group es un diccionario de Python con los listas numéricas que representan
    elementos del grupo de permutación.
    """
    g3 = []
    if type(gi) != type('String') or type(gj) != type('String') :
        print('¡Error! Los elementos del grupo deben ser tipo string.')
        return g3
    if type(group) != type({}):
        print('¡Error! El grupo de permutaciones debe estar almacenado en un diccionario de Python.')
        return g3
    if not gi in group.keys():
        print('¡Error! %s no es un elemento del grupo.' % gi)
        return g3
    if not gj in group.keys():
        print('¡Error! %s no es un elemento del grupo.' % gj)
        return g3
    g1 = group[gi]
    g2 = group[gj]
    if len(g1) != len(g2):
        print('¡Error! Los elementos del grupo deben tener el mismo tamaño. len(gi)=%d, len(gj)=%d' % (len(gi), len(gj)))
        return g3
    g3 = [g2[i-1] for i in g1]
    return g3

In [7]:
def get_element_name(g, group):
    """
    get_element_name(g) retorna la etiqueta (string) del elemento g (lista)
    del grupo group.
    """
    if type(g) == type('String') and g in group.keys():
        g = group[g]
    for name, element in group.items():
        if g == element:
            return name 
    print('¡Advertencia! El elemento no pertenece al grupo.')
    return None

In [8]:
def prod_name(gi, gj, group):
    """
    prod_name(gi, gj, group) retorna el nombre del elemento resultante del producto gi gj.
    """
    return get_element_name(prod(gi, gj, group), group)

In [9]:
def get_inverse(g, group):
    """
    get_inverse(g, group) retorna el elemento inverso de g.
    """
    elements = list(group.keys())
    g_value = list(group.values())[0]
    identity = [i+1 for i in range(len(g_value))]
    for gi in elements:
        gk = prod(g, gi, group)
        if identity == gk:
            return get_element_name(str(gi), group)
    return 'null'

In [10]:
def get_conjugate(g, x, group):
    """
    get_conjugate(g, x, group) devuelve el elemento conjugado a g obtenido como x g x^-1
    """
    x_inv = get_inverse(x, group)
    xg = prod_name(x, g, group)
    return prod_name(xg, x_inv, group)

In [11]:
def get_class(group):
    """
    get_class(group) retorna los elementos de las clases del grupo group.
    """
    elements = list(group.keys())
    inverse_elements = dict(zip(elements, [get_inverse(g, group) for g in elements]))
    in_class = elements.copy()
    class_elements = {}
    c = 0
    while len(in_class) > 0:
        c += 1
        g = in_class[0]
        in_class.remove(g)
        this_class_element = list(set([get_conjugate(g, x, group) for 
        x in elements]))
        in_class = [x for x in in_class if x not in this_class_element]
        class_elements['C%d' % c] = this_class_element
    return class_elements
        

In [12]:
def get_subgroups(group):
    """
    get_subgroups(group) retorna todos los subgrupos diferentes obtenidos al multiplicar consecutivamente un elemento g por si mismo.
    """
    keys = []
    subgrups = []
    s = 0
    for g in group.keys():
        g0 = g
        gn = [g0]
        for n in range(len(group)):
            gn.append(prod_name(g, gn[-1], group))
        elements = sorted(list(set(gn)))
        if not elements in subgrups:
            s += 1
            keys.append('S%d' % s)
            subgrups.append(elements)
    return dict(zip(keys, subgrups))

In [13]:
def get_right_cosets(subgroup, group):
    """
    get_right_cosets(subgroup, group) retorna todos los co-conjuntos derechos del subgrupo obtenidos para cada elemento del grupo.
    """
    right_cosets = {}
    for x in group.keys():
        right_cosets[x] = [prod_name(g, x, group) for g in subgroup]
    return right_cosets

In [14]:
def get_left_cosets(subgroup, group):
    """
    get_left_cosets(subgroup, group) retorna todos los co-conjuntos izquierdos del subgrupo obtenidos para cada elemento del grupo.
    """
    left_cosets = {}
    for x in group.keys():
        left_cosets[x] = [prod_name(x, g, group) for g in subgroup]
    return left_cosets

## 4. Tabla de multiplicación

4.1. Multiplicación de dos elementos del grupo

In [15]:
group = P4
gi_name = 'a'
gj_name = 'b'
gk = prod(gi_name, gj_name, group)
gk_name = get_element_name(gk, group)
print('%s %s = %s' % (gi_name, gj_name, gk_name))

a b = c


In [16]:
elem_names = list(group.keys())
names  = pd.DataFrame(columns=elem_names, index=elem_names)
values = pd.DataFrame(columns=elem_names, index=elem_names)
for gi in group.keys():
    for gj in group.keys():
        gk_val = prod(gi, gj, group)
        gk = get_element_name(gk_val, group)
        names.loc[gi, gj] = gk
        values.loc[gi, gj] = gk_val


In [17]:
group

{'e': [1, 2, 3, 4],
 'a': [1, 2, 4, 3],
 'b': [2, 1, 3, 4],
 'c': [2, 1, 4, 3],
 'd': [1, 3, 2, 4],
 'f': [1, 3, 4, 2],
 'g': [3, 1, 2, 4],
 'h': [3, 1, 4, 2],
 'i': [2, 3, 1, 4],
 'j': [2, 3, 4, 1],
 'k': [3, 2, 1, 4],
 'l': [3, 2, 4, 1],
 'm': [1, 4, 2, 3],
 'n': [1, 4, 3, 2],
 'o': [4, 1, 2, 3],
 'p': [4, 1, 3, 2],
 'q': [2, 4, 1, 3],
 'r': [2, 4, 3, 1],
 's': [4, 2, 1, 3],
 't': [4, 2, 3, 1],
 'u': [3, 4, 1, 2],
 'v': [3, 4, 2, 1],
 'w': [4, 3, 1, 2],
 'y': [4, 3, 2, 1]}

In [18]:
names

,e,a,b,c,d,f,g,h,i,j,...,o,p,q,r,s,t,u,v,w,y
e,e,a,b,c,d,f,g,h,i,j,...,o,p,q,r,s,t,u,v,w,y
a,a,e,c,b,f,d,h,g,j,i,...,p,o,r,q,t,s,v,u,y,w
b,b,c,e,a,g,h,d,f,k,l,...,m,n,s,t,q,r,w,y,u,v
c,c,b,a,e,h,g,f,d,l,k,...,n,m,t,s,r,q,y,w,v,u
d,d,m,i,q,e,n,k,u,b,r,...,s,w,c,j,o,y,h,l,p,t
f,f,n,j,r,a,m,l,v,c,q,...,t,y,b,i,p,w,g,k,o,s
g,g,o,k,s,b,p,i,w,e,t,...,q,u,a,l,m,v,f,j,n,r
h,h,p,l,t,c,o,j,y,a,s,...,r,v,e,k,n,u,d,i,m,q
i,i,q,d,m,k,u,e,n,g,v,...,a,f,o,y,c,j,p,t,h,l
j,j,r,f,n,l,v,a,m,h,u,...,e,d,p,w,b,i,o,s,g,k


In [19]:
values

,e,a,b,c,d,f,g,h,i,j,...,o,p,q,r,s,t,u,v,w,y
e,"[1, 2, 3, 4]","[1, 2, 4, 3]","[2, 1, 3, 4]","[2, 1, 4, 3]","[1, 3, 2, 4]","[1, 3, 4, 2]","[3, 1, 2, 4]","[3, 1, 4, 2]","[2, 3, 1, 4]","[2, 3, 4, 1]",...,"[4, 1, 2, 3]","[4, 1, 3, 2]","[2, 4, 1, 3]","[2, 4, 3, 1]","[4, 2, 1, 3]","[4, 2, 3, 1]","[3, 4, 1, 2]","[3, 4, 2, 1]","[4, 3, 1, 2]","[4, 3, 2, 1]"
a,"[1, 2, 4, 3]","[1, 2, 3, 4]","[2, 1, 4, 3]","[2, 1, 3, 4]","[1, 3, 4, 2]","[1, 3, 2, 4]","[3, 1, 4, 2]","[3, 1, 2, 4]","[2, 3, 4, 1]","[2, 3, 1, 4]",...,"[4, 1, 3, 2]","[4, 1, 2, 3]","[2, 4, 3, 1]","[2, 4, 1, 3]","[4, 2, 3, 1]","[4, 2, 1, 3]","[3, 4, 2, 1]","[3, 4, 1, 2]","[4, 3, 2, 1]","[4, 3, 1, 2]"
b,"[2, 1, 3, 4]","[2, 1, 4, 3]","[1, 2, 3, 4]","[1, 2, 4, 3]","[3, 1, 2, 4]","[3, 1, 4, 2]","[1, 3, 2, 4]","[1, 3, 4, 2]","[3, 2, 1, 4]","[3, 2, 4, 1]",...,"[1, 4, 2, 3]","[1, 4, 3, 2]","[4, 2, 1, 3]","[4, 2, 3, 1]","[2, 4, 1, 3]","[2, 4, 3, 1]","[4, 3, 1, 2]","[4, 3, 2, 1]","[3, 4, 1, 2]","[3, 4, 2, 1]"
c,"[2, 1, 4, 3]","[2, 1, 3, 4]","[1, 2, 4, 3]","[1, 2, 3, 4]","[3, 1, 4, 2]","[3, 1, 2, 4]","[1, 3, 4, 2]","[1, 3, 2, 4]","[3, 2, 4, 1]","[3, 2, 1, 4]",...,"[1, 4, 3, 2]","[1, 4, 2, 3]","[4, 2, 3, 1]","[4, 2, 1, 3]","[2, 4, 3, 1]","[2, 4, 1, 3]","[4, 3, 2, 1]","[4, 3, 1, 2]","[3, 4, 2, 1]","[3, 4, 1, 2]"
d,"[1, 3, 2, 4]","[1, 4, 2, 3]","[2, 3, 1, 4]","[2, 4, 1, 3]","[1, 2, 3, 4]","[1, 4, 3, 2]","[3, 2, 1, 4]","[3, 4, 1, 2]","[2, 1, 3, 4]","[2, 4, 3, 1]",...,"[4, 2, 1, 3]","[4, 3, 1, 2]","[2, 1, 4, 3]","[2, 3, 4, 1]","[4, 1, 2, 3]","[4, 3, 2, 1]","[3, 1, 4, 2]","[3, 2, 4, 1]","[4, 1, 3, 2]","[4, 2, 3, 1]"
f,"[1, 3, 4, 2]","[1, 4, 3, 2]","[2, 3, 4, 1]","[2, 4, 3, 1]","[1, 2, 4, 3]","[1, 4, 2, 3]","[3, 2, 4, 1]","[3, 4, 2, 1]","[2, 1, 4, 3]","[2, 4, 1, 3]",...,"[4, 2, 3, 1]","[4, 3, 2, 1]","[2, 1, 3, 4]","[2, 3, 1, 4]","[4, 1, 3, 2]","[4, 3, 1, 2]","[3, 1, 2, 4]","[3, 2, 1, 4]","[4, 1, 2, 3]","[4, 2, 1, 3]"
g,"[3, 1, 2, 4]","[4, 1, 2, 3]","[3, 2, 1, 4]","[4, 2, 1, 3]","[2, 1, 3, 4]","[4, 1, 3, 2]","[2, 3, 1, 4]","[4, 3, 1, 2]","[1, 2, 3, 4]","[4, 2, 3, 1]",...,"[2, 4, 1, 3]","[3, 4, 1, 2]","[1, 2, 4, 3]","[3, 2, 4, 1]","[1, 4, 2, 3]","[3, 4, 2, 1]","[1, 3, 4, 2]","[2, 3, 4, 1]","[1, 4, 3, 2]","[2, 4, 3, 1]"
h,"[3, 1, 4, 2]","[4, 1, 3, 2]","[3, 2, 4, 1]","[4, 2, 3, 1]","[2, 1, 4, 3]","[4, 1, 2, 3]","[2, 3, 4, 1]","[4, 3, 2, 1]","[1, 2, 4, 3]","[4, 2, 1, 3]",...,"[2, 4, 3, 1]","[3, 4, 2, 1]","[1, 2, 3, 4]","[3, 2, 1, 4]","[1, 4, 3, 2]","[3, 4, 1, 2]","[1, 3, 2, 4]","[2, 3, 1, 4]","[1, 4, 2, 3]","[2, 4, 1, 3]"
i,"[2, 3, 1, 4]","[2, 4, 1, 3]","[1, 3, 2, 4]","[1, 4, 2, 3]","[3, 2, 1, 4]","[3, 4, 1, 2]","[1, 2, 3, 4]","[1, 4, 3, 2]","[3, 1, 2, 4]","[3, 4, 2, 1]",...,"[1, 2, 4, 3]","[1, 3, 4, 2]","[4, 1, 2, 3]","[4, 3, 2, 1]","[2, 1, 4, 3]","[2, 3, 4, 1]","[4, 1, 3, 2]","[4, 2, 3, 1]","[3, 1, 4, 2]","[3, 2, 4, 1]"
j,"[2, 3, 4, 1]","[2, 4, 3, 1]","[1, 3, 4, 2]","[1, 4, 3, 2]","[3, 2, 4, 1]","[3, 4, 2, 1]","[1, 2, 4, 3]","[1, 4, 2, 3]","[3, 1, 4, 2]","[3, 4, 1, 2]",...,"[1, 2, 3, 4]","[1, 3, 2, 4]","[4, 1, 3, 2]","[4, 3, 1, 2]","[2, 1, 3, 4]","[2, 3, 1, 4]","[4, 1, 2, 3]","[4, 2, 1, 3]","[3, 1, 2, 4]","[3, 2, 1, 4]"


## 5. Classes

Los elementos de una clase corresponden a todos aquellos que son obtenidos por operaciones de conjugación.

In [20]:
group_class = get_class(group)
group_class

{'C1': ['e'],
 'C2': ['n', 'k', 'a', 'd', 'b', 't'],
 'C3': ['u', 'c', 'y'],
 'C4': ['m', 'l', 'g', 'f', 'p', 's', 'r', 'i'],
 'C5': ['o', 'h', 'w', 'j', 'q', 'v']}

## 6. Sugrupos

Una manera de obtener subgrupos es tomar un cierto elemento g y multiplicarlo consecutivamente por si mismo.  Todos los elementos obtenidos de esta manera conrresponderán con un subgrupo del grupo.

In [21]:
subgroups = get_subgroups(group)
subgroups

{'S1': ['e'],
 'S2': ['a', 'e'],
 'S3': ['b', 'e'],
 'S4': ['c', 'e'],
 'S5': ['d', 'e'],
 'S6': ['e', 'f', 'm'],
 'S7': ['e', 'g', 'i'],
 'S8': ['e', 'h', 'q', 'y'],
 'S9': ['e', 'j', 'o', 'u'],
 'S10': ['e', 'k'],
 'S11': ['e', 'l', 's'],
 'S12': ['e', 'n'],
 'S13': ['e', 'p', 'r'],
 'S14': ['e', 't'],
 'S15': ['e', 'u'],
 'S16': ['c', 'e', 'v', 'w'],
 'S17': ['e', 'y']}

## 7. Co-conjuntos de los subgrupos

Los co-conjuntos de un subgrupo se obtienen multiplicando un elemento $x$ del grupo por todos los elementos $g$ del subgrupo.  Hay dos posibilidades:

+ Co-conjunto derecho: cuando el producto entre elementos es de la forma $gx$.
+ Co-conjunto izquierdo: cuando el producto entre elementos es de la forma $xg$.

In [22]:
s = 'S8'
right_cosets = get_right_cosets(subgroups[s], group)
right_cosets

{'e': ['e', 'h', 'q', 'y'],
 'a': ['a', 'p', 'i', 'v'],
 'b': ['b', 'l', 'm', 'w'],
 'c': ['c', 't', 'd', 'u'],
 'd': ['d', 'c', 'u', 't'],
 'f': ['f', 'o', 'k', 'r'],
 'g': ['g', 'j', 'n', 's'],
 'h': ['h', 'y', 'e', 'q'],
 'i': ['i', 'a', 'v', 'p'],
 'j': ['j', 's', 'g', 'n'],
 'k': ['k', 'f', 'r', 'o'],
 'l': ['l', 'w', 'b', 'm'],
 'm': ['m', 'b', 'w', 'l'],
 'n': ['n', 'g', 's', 'j'],
 'o': ['o', 'r', 'f', 'k'],
 'p': ['p', 'v', 'a', 'i'],
 'q': ['q', 'e', 'y', 'h'],
 'r': ['r', 'k', 'o', 'f'],
 's': ['s', 'n', 'j', 'g'],
 't': ['t', 'u', 'c', 'd'],
 'u': ['u', 'd', 't', 'c'],
 'v': ['v', 'i', 'p', 'a'],
 'w': ['w', 'm', 'l', 'b'],
 'y': ['y', 'q', 'h', 'e']}

In [23]:
s = 'S8'
left_cosets = get_left_cosets(subgroups[s], group)
left_cosets

{'e': ['e', 'h', 'q', 'y'],
 'a': ['a', 'g', 'r', 'w'],
 'b': ['b', 'f', 's', 'v'],
 'c': ['c', 'd', 't', 'u'],
 'd': ['d', 'u', 'c', 't'],
 'f': ['f', 'v', 'b', 's'],
 'g': ['g', 'w', 'a', 'r'],
 'h': ['h', 'y', 'e', 'q'],
 'i': ['i', 'n', 'o', 'l'],
 'j': ['j', 'm', 'p', 'k'],
 'k': ['k', 'p', 'm', 'j'],
 'l': ['l', 'o', 'n', 'i'],
 'm': ['m', 'k', 'j', 'p'],
 'n': ['n', 'l', 'i', 'o'],
 'o': ['o', 'i', 'l', 'n'],
 'p': ['p', 'j', 'k', 'm'],
 'q': ['q', 'e', 'y', 'h'],
 'r': ['r', 'a', 'w', 'g'],
 's': ['s', 'b', 'v', 'f'],
 't': ['t', 'c', 'u', 'd'],
 'u': ['u', 't', 'd', 'c'],
 'v': ['v', 's', 'f', 'b'],
 'w': ['w', 'r', 'g', 'a'],
 'y': ['y', 'q', 'h', 'e']}

End!